In [18]:
%load_ext sql
%sql sqlite:///StockQuotes.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Get company IDs

In [19]:
res = %sql SELECT company, ticker FROM Company;
res

 * sqlite:///StockQuotes.db
Done.


company,ticker
Papa John's,PZZA
Domino's Pizza,DPZ


Date Range by Company

In [20]:
PAPA_JOHN = 1

sqs = %sql SELECT company, ticker, date, close \
FROM Stock_Quotes \
INNER JOIN Company ON Stock_Quotes.company_id = company.id \
WHERE company_id = {PAPA_JOHN} \
AND date(date) >= date('2019-06-01') AND date <= '2019-06-30' \
ORDER BY date

sqs

 * sqlite:///StockQuotes.db
Done.


company,ticker,date,close
Papa John's,PZZA,2019-06-03,48.669998
Papa John's,PZZA,2019-06-04,48.389999
Papa John's,PZZA,2019-06-05,49.279999
Papa John's,PZZA,2019-06-06,50.59
Papa John's,PZZA,2019-06-07,50.580002
Papa John's,PZZA,2019-06-10,49.200001
Papa John's,PZZA,2019-06-11,49.41
Papa John's,PZZA,2019-06-12,49.349998
Papa John's,PZZA,2019-06-13,50.5
Papa John's,PZZA,2019-06-14,50.740002


Like a `.describe()` without *standard deviation* for 2019.

In [21]:
agg_res = %sql SELECT \
    company, \
    ticker, \
    COUNT(close) AS 'close_count', \
    AVG(close) AS 'close_avg', \
    MIN(close) AS 'close_min', \
    MAX(close) AS 'close_max' \
FROM Stock_Quotes \
INNER JOIN Company ON Stock_Quotes.company_id = company.id \
WHERE company_id = {PAPA_JOHN} \
AND date(date) >= date('2019-01-01' ) AND date(date) <= date('2019-12-31');

agg_res

 * sqlite:///StockQuotes.db
Done.


company,ticker,close_count,close_avg,close_min,close_max
Papa John's,PZZA,252,50.01876981349208,38.509998,65.040001


Get **Avg, Min, Max** for ***Open High, Low, Close***.

In [22]:
def sql_union(aggs_list, COMPANY):
    """Unionize open, high, low, close."""
    res_des = ''
    for agg in aggs_list:
        agg_type = agg.lower()
        print(agg, agg_type)
        res_des += f"SELECT company, ticker, '{agg}', COUNT({agg_type}) AS 'Count', AVG({agg_type}) AS 'Avg', MIN({agg_type}) AS 'Min', MAX({agg_type}) AS 'Max' FROM Stock_Quotes INNER JOIN Company ON Stock_Quotes.company_id = company.id WHERE company_id = {COMPANY} UNION ALL "
    return res_des

aggs_list = ['Open', 'High', 'Low', 'Close']
the_sql = sql_union(aggs_list, PAPA_JOHN)

the_sql = the_sql[:-10].strip().replace('"', '')
print(the_sql)

res = %sql SELECT company, ticker, 'Open', COUNT(open) AS 'Count', AVG(open) AS 'Avg', MIN(open) AS 'Min', MAX(open) AS 'Max' FROM Stock_Quotes INNER JOIN Company ON Stock_Quotes.company_id = company.id WHERE company_id = 1 UNION ALL SELECT company, ticker, 'High', COUNT(high) AS 'Count', AVG(high) AS 'Avg', MIN(high) AS 'Min', MAX(high) AS 'Max' FROM Stock_Quotes INNER JOIN Company ON Stock_Quotes.company_id = company.id WHERE company_id = 1 UNION ALL SELECT company, ticker, 'Low', COUNT(low) AS 'Count', AVG(low) AS 'Avg', MIN(low) AS 'Min', MAX(low) AS 'Max' FROM Stock_Quotes INNER JOIN Company ON Stock_Quotes.company_id = company.id WHERE company_id = 1 UNION ALL SELECT company, ticker, 'Close', COUNT(close) AS 'Count', AVG(close) AS 'Avg', MIN(close) AS 'Min', MAX(close) AS 'Max' FROM Stock_Quotes INNER JOIN Company ON Stock_Quotes.company_id = company.id WHERE company_id = 1
res

Open open
High high
Low low
Close close
SELECT company, ticker, 'Open', COUNT(open) AS 'Count', AVG(open) AS 'Avg', MIN(open) AS 'Min', MAX(open) AS 'Max' FROM Stock_Quotes INNER JOIN Company ON Stock_Quotes.company_id = company.id WHERE company_id = 1 UNION ALL SELECT company, ticker, 'High', COUNT(high) AS 'Count', AVG(high) AS 'Avg', MIN(high) AS 'Min', MAX(high) AS 'Max' FROM Stock_Quotes INNER JOIN Company ON Stock_Quotes.company_id = company.id WHERE company_id = 1 UNION ALL SELECT company, ticker, 'Low', COUNT(low) AS 'Count', AVG(low) AS 'Avg', MIN(low) AS 'Min', MAX(low) AS 'Max' FROM Stock_Quotes INNER JOIN Company ON Stock_Quotes.company_id = company.id WHERE company_id = 1 UNION ALL SELECT company, ticker, 'Close', COUNT(close) AS 'Count', AVG(close) AS 'Avg', MIN(close) AS 'Min', MAX(close) AS 'Max' FROM Stock_Quotes INNER JOIN Company ON Stock_Quotes.company_id = company.id WHERE company_id = 1
 * sqlite:///StockQuotes.db
Done.


company,ticker,'Open',Count,Avg,Min,Max
Papa John's,PZZA,Open,7288,28.139977331366648,2.0,138.3
Papa John's,PZZA,High,7288,28.557411310373226,2.055556,140.68
Papa John's,PZZA,Low,7288,27.730881552963723,1.888889,134.57
Papa John's,PZZA,Close,7288,28.14848251454453,1.944444,140.01


### Find Duplicates

In [23]:
dups_res = %sql SELECT company, ticker, date, `open`, high, low, close, adj_close, COUNT(date) AS date_ct \
FROM Stock_Quotes \
INNER JOIN Company ON company.id = Stock_Quotes.company_id \
WHERE company_id = 1 \
GROUP BY date \
HAVING date_ct > 1 \
ORDER BY date_ct DESC

dups_res

 * sqlite:///StockQuotes.db
Done.


company,ticker,date,open,high,low,close,adj_close,date_ct
Papa John's,PZZA,2021-12-10,130.75,130.97,127.78,128.34,128.34,2
Papa John's,PZZA,2021-11-29,128.9,130.19,126.0,126.21,126.21,2
Papa John's,PZZA,2021-11-26,132.42,135.8,128.37,128.84,128.84,2
Papa John's,PZZA,2021-11-24,128.53,133.95,128.53,133.84,133.84,2
Papa John's,PZZA,2021-11-23,130.53,132.23,128.22,130.52,130.52,2
Papa John's,PZZA,2021-11-22,132.54,134.37,129.9,130.5,130.5,2
Papa John's,PZZA,2021-11-19,131.78,133.98,131.59,133.06,133.06,2
Papa John's,PZZA,2021-11-18,129.83,132.57,128.08,131.78,131.78,2
Papa John's,PZZA,2021-11-17,128.88,130.8,128.01,129.64,129.64,2
Papa John's,PZZA,2021-11-16,130.86,131.37,128.71,129.31,129.31,2


### Duplicates
If we query by one of the duplicate dates we can see that teh `adj_close` is different. This is because data was imported from 2 different sources.

In [24]:
dup_res = %sql SELECT * from Stock_Quotes WHERE date = '2021-12-10' AND company_id={PAPA_JOHN};
dup_res

 * sqlite:///StockQuotes.db
Done.


id,str_date,date,open,high,low,close,adj_close,volume,created,updated,company_id
11605,"Dec 10, 2021",2021-12-10,130.75,130.97,127.78,128.34,128.34,189900,2021-12-14 16:29:53.448412-05,2021-12-14 16:29:53.453135-05,1
11781,"Dec 10, 2021",2021-12-10,130.75,130.97,127.78,128.34,127.97,189900,2022-04-13 14:57:14.515316-04,2022-04-13 14:57:14.518725-04,1


### N Tiles

In [25]:
# N tile

BUCKETS = 5

tile_res = %sql SELECT close, NTILE( {BUCKETS} ) OVER ( \
    ORDER BY close \
)  LengthOfBucket \
FROM Stock_Quotes \
WHERE company_id = {PAPA_JOHN} \
AND date(date) >= date('2019-01-01') AND date < '2019-02-01';

tile_res

 * sqlite:///StockQuotes.db
Done.


close,LengthOfBucket
40.130001,1
40.299999,1
41.630001,1
41.740002,1
42.139999,1
42.18,2
42.200001,2
42.290001,2
42.330002,2
42.419998,3


A varition of the above N tiles query

In [26]:
tile2_res = %sql SELECT close, NTILE(5) OVER ( \
    PARTITION BY company_id \
    ORDER BY close \
)  LengthOfBucket \
FROM Stock_Quotes \
WHERE company_id = 1 \
AND date(date) >= date('2019-01-01') AND date < '2019-02-01';

tile2_res

 * sqlite:///StockQuotes.db
Done.


close,LengthOfBucket
40.130001,1
40.299999,1
41.630001,1
41.740002,1
42.139999,1
42.18,2
42.200001,2
42.290001,2
42.330002,2
42.419998,3


### Percentage Groups

In [27]:
# Percentage (%)
percent_dict = {
    '25': 2.5,
    '50': 5.0,
    '75': 7.5
}

for prct_k, prct_v in percent_dict.items():
    #print(prct_k, prct_v)
    per_res = %sql SELECT close AS '{prct_k}% of close price' \
    FROM Stock_Quotes WHERE company_id = {PAPA_JOHN} \
    ORDER BY close ASC \
    LIMIT 1 \
    OFFSET ( \
        SELECT COUNT(close) FROM Stock_Quotes  WHERE company_id = {PAPA_JOHN} \
    ) * {prct_v} / 10 - 1

    print(per_res)

 * sqlite:///StockQuotes.db
Done.
+--------------------+
| 25% of close price |
+--------------------+
|       7.4875       |
+--------------------+
 * sqlite:///StockQuotes.db
Done.
+--------------------+
| 50% of close price |
+--------------------+
|       12.755       |
+--------------------+
 * sqlite:///StockQuotes.db
Done.
+--------------------+
| 75% of close price |
+--------------------+
|       46.18        |
+--------------------+


### Percent_rank

In [28]:
per_ra_res = %sql SELECT date, close, PERCENT_RANK() OVER ( \
    ORDER BY close \
) LengthPercentRank \
FROM Stock_Quotes \
WHERE company_id = {PAPA_JOHN} \
AND date >= date('2020-01-01') AND date(date) < date('2020-02-01') \
ORDER BY LengthPercentRank DESC;

per_ra_res

 * sqlite:///StockQuotes.db
Done.


date,close,LengthPercentRank
2020-01-23,66.919998,1.0
2020-01-24,66.529999,0.95
2020-01-22,66.139999,0.9
2020-01-28,66.050003,0.85
2020-01-29,66.010002,0.8
2020-01-30,65.440002,0.75
2020-01-09,65.279999,0.7
2020-01-16,65.239998,0.65
2020-01-17,65.059998,0.6
2020-01-27,64.940002,0.55


To be continued...